# Submission 1: Twitter Sentiment Analysis
Nama: Muhammad Ridwan Ramadhan

Username dicoding: muhammad_ridwan_07rg

| | Deskripsi |
| ----------- | ----------- |
| Dataset | [tweets sentiment analysis](https://www.kaggle.com/datasets/youben/tweets-sentiment-analysis/data) |
| Masalah | Penentuan sentimen dari tweets yang beredar di twitter |
| Solusi machine learning | NLP |
| Metode pengolahan | Vektorisasi  |
| Arsitektur model | Deskripsi arsitektur model yang diguanakan |
| Metrik evaluasi | accuracy,confusion matrix,AUC,F-1 Score  |
| Performa model | Deksripsi performa model yang dibuat |

In [1]:
import tensorflow as tf
import os
import tensorflow_model_analysis as tfma 
from tfx.components import CsvExampleGen,StatisticsGen,SchemaGen,ExampleValidator,Transform,Trainer,Evaluator,InfraValidator,Pusher, Tuner
from tfx.proto import example_gen_pb2,trainer_pb2, pusher_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.dsl.components.common.resolver import Resolver 
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy 
from tfx.types import Channel 
from tfx.types.standard_artifacts import Model, ModelBlessing 
from tfx.v1.proto import ServingSpec, TensorFlowServing, LocalDockerConfig

In [2]:
PIPELINE_NAME='tweet-pipeline'

SCHEMA_PIPELINE_NAME='tweet_schema'

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)



In [3]:
data='data/'

output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=7),
        example_gen_pb2.SplitConfig.Split(name="test", hash_buckets=1),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)
example_gen = CsvExampleGen(input_base=data, output_config=output)

In [4]:
interactive_context=InteractiveContext(PIPELINE_NAME,PIPELINE_ROOT)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [5]:
statistics_gen=StatisticsGen(examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [6]:
interactive_context.show(statistics_gen.outputs['statistics'])

In [7]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Sentiment',INT,required,,-
'SentimentText',BYTES,required,,-


In [9]:
example_validator=ExampleValidator(schema=schema_gen.outputs['schema'],
                                   statistics=statistics_gen.outputs['statistics'])

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
interactive_context.show(example_validator.outputs['anomalies'])

In [11]:
MODULE_NAME='tweet_preprocessing.py'

In [12]:
%%writefile {MODULE_NAME}
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY='Sentiment'
FEATURE_KEY='SentimentText'


def transformed_name(key):
    return key+'_xf'

def preprocessing_fn(inputs):
  outputs = {}  
  outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
  outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
  return outputs

Overwriting tweet_preprocessing.py


In [13]:
transform=Transform(examples=example_gen.outputs['examples'],
                    schema=schema_gen.outputs['schema'],
                    module_file=os.path.abspath(MODULE_NAME))

interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\tweet-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\4ee26428aaac49f0aaa2990efbeb58c1\assets


INFO:tensorflow:Assets written to: pipelines\tweet-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\4ee26428aaac49f0aaa2990efbeb58c1\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [14]:
MODULE_TRAINER='tweet_trainer.py'

In [15]:
%%writefile {MODULE_TRAINER}
import tensorflow as tf
import tensorflow_transform as tft 
from tensorflow.keras import layers
import os  
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs
 
LABEL_KEY = "Sentiment"
FEATURE_KEY = "SentimentText"
 
def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"
 
def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')
 
 
def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=512)->tf.data.Dataset:
    
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())
    
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 

VOCAB_SIZE = 5000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder():
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    
    model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder()
    
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            epochs=20)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting tweet_trainer.py


In [16]:
trainer  = Trainer(
    module_file=os.path.abspath(MODULE_TRAINER),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['test'])
)
interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SentimentText_xf (InputLay  [(None, 1)]               0         
 er)                                                             
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           80000     
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                             

INFO:tensorflow:Assets written to: pipelines\tweet-pipeline\Trainer\model\6\Format-Serving\assets


1365/1365 [==============================] - 78s 54ms/step - loss: 0.4668 - binary_accuracy: 0.7684 - val_loss: 0.5300 - val_binary_accuracy: 0.7558
Epoch 2/20
1364/1365 [============================>.] - ETA: 0s - loss: 0.4184 - binary_accuracy: 0.7950
Epoch 2: val_binary_accuracy did not improve from 0.75581
1365/1365 [==============================] - 47s 34ms/step - loss: 0.4183 - binary_accuracy: 0.7950 - val_loss: 0.5964 - val_binary_accuracy: 0.7472
Epoch 3/20
1364/1365 [============================>.] - ETA: 0s - loss: 0.3717 - binary_accuracy: 0.8260
Epoch 3: val_binary_accuracy did not improve from 0.75581
1365/1365 [==============================] - 48s 35ms/step - loss: 0.3717 - binary_accuracy: 0.8260 - val_loss: 0.6475 - val_binary_accuracy: 0.7367
Epoch 4/20
1362/1365 [============================>.] - ETA: 0s - loss: 0.3227 - binary_accuracy: 0.8497
Epoch 4: val_binary_accuracy did not improve from 0.75581
1365/1365 [==============================] - 42s 31ms/step - los

...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\tweet-pipeline\Trainer\model\6\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\tweet-pipeline\Trainer\model\6\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [17]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
 
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [18]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='Sentiment')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[
            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.8}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]
 
)

In [19]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
 
interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [20]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

In [21]:
TUNER_MODULE='tuner.py'

In [22]:
%%writefile {TUNER_MODULE}

import tensorflow as tf
from collections import namedtuple
from keras_tuner.engine import base_tuner
from keras_tuner.tuners import Hyperband
from typing import Dict,Text,Any
frm tfx.components.trainer.fn_args_utils import FnArgs

TunerFnResult = namedtuple('TunerFnResult', [('tuner', base_tuner.BaseTuner),
                                             ('fit_kwargs', Dict[Text, Any])])


FEATURE_KEY='SentimentText'

def model_builder(hp):
  hp_nodes=hp.Int('units',min_value=64,max_value=256,step=64)
  hp_learning_rate=hp.Choice('learning_rate',values=[1e-2,1e-3,1e-4])
  hp_vocab=hp.Int('input_dim',values=[x for x in range(1000,10001,1000)])

  embedding_dim=16

  inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
  reshaped_narrative = tf.reshape(inputs, [-1])
  x = vectorize_layer(reshaped_narrative)
  x = layers.Embedding(hp_vocab, embedding_dim, name="embedding")(x)
  x = layers.GlobalAveragePooling1D()(x)
  x = layers.Dense(hp_nodes, activation='relu')(x)
  x = layers.Dense(32, activation="relu")(x)
  outputs = layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs=inputs, outputs = outputs)

  model.compile(
      loss = 'binary_crossentropy',
      optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
      metrics=[tf.keras.metrics.BinaryAccuracy()])

  return model

def tuner_fn(fn_args: FnArgs) -> TunerFnResult:
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = input_fn(fn_args.train_files, fn_args.data_accessor, tf_transform_output)
  eval_dataset = input_fn(fn_args.eval_files, fn_args.data_accessor, tf_transform_output)

  tuner=Hyperband(model_builder,
                  objective='val_accuracy',
                  max_epochs=10,
                  directory=fn_args.working_dir,
                  project_name='sentimentanalyzer_tuner')

  return TunerFnResult(tuner=tuner,fit_kwargs={'x':train_dataset,'validation_data':val_dataset})

Overwriting tuner.py


In [23]:
tuner=Tuner(
    module_file=os.path.abspath(TUNER_MODULE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'])
)

In [24]:
infra_validator=InfraValidator(model=trainer.outputs['model'],
                               serving_spec=ServingSpec(tensorflow_serving=TensorFlowServing(tags=['latest']),
                                                        local_docker=LocalDockerConfig()))

In [25]:
pusher=Pusher(model=trainer.outputs['model'],
              model_blessing=evaluator.outputs['blessing'],
              infra_blessing=infra_validator.outputs['blessing'],
              push_destination=pusher_pb2.PushDestination()
              )

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)